In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
data1 = pd.read_csv('datasets_11796_16250_ionosphere_data_kaggle.csv')
data1.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,label
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351 entries, 0 to 350
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature1   351 non-null    int64  
 1   feature2   351 non-null    int64  
 2   feature3   351 non-null    float64
 3   feature4   351 non-null    float64
 4   feature5   351 non-null    float64
 5   feature6   351 non-null    float64
 6   feature7   351 non-null    float64
 7   feature8   351 non-null    float64
 8   feature9   351 non-null    float64
 9   feature10  351 non-null    float64
 10  feature11  351 non-null    float64
 11  feature12  351 non-null    float64
 12  feature13  351 non-null    float64
 13  feature14  351 non-null    float64
 14  feature15  351 non-null    float64
 15  feature16  351 non-null    float64
 16  feature17  351 non-null    float64
 17  feature18  351 non-null    float64
 18  feature19  351 non-null    float64
 19  feature20  351 non-null    float64
 20  feature21 

In [4]:
data1["label"].value_counts()

g    225
b    126
Name: label, dtype: int64

In [5]:
# Stratified Sampling using Scikit-learn's Stratified Shuffle Split Class
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=42)
for train_index, test_index in split.split(data1, data1["label"]):
    strat_train_set = data1.loc[train_index]
    strat_test_set = data1.loc[test_index]

In [6]:
 strat_test_set["label"].value_counts()

g    56
b    32
Name: label, dtype: int64

In [7]:
strat_train_set["label"].value_counts()

g    169
b     94
Name: label, dtype: int64

In [8]:
train_set = strat_train_set.drop("label", axis=1) # drop labels for training set
train_labels = strat_train_set["label"].copy()
test_set = strat_test_set.drop("label", axis=1) # drop labels for testing set
test_labels = strat_test_set["label"].copy()

In [9]:
train_labels

79     b
312    g
46     g
218    b
344    g
      ..
10     g
217    g
71     b
134    b
193    g
Name: label, Length: 263, dtype: object

In [10]:
test_labels

181    g
122    b
186    b
285    g
128    b
      ..
29     b
238    b
177    g
144    b
139    g
Name: label, Length: 88, dtype: object

In [11]:
from sklearn.metrics import matthews_corrcoef
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV

parameters = {'solver':["svd","lsqr"],
              "priors":[None],
              "n_components": [None],
              "tol": [1.0e-4]
             }
clf2 = LDA()

clf = GridSearchCV(clf2, parameters, scoring = 'balanced_accuracy', cv=10)
res1=clf.fit(train_set, train_labels)
res1.best_estimator_

LinearDiscriminantAnalysis()

In [12]:
res1.best_params_

{'n_components': None, 'priors': None, 'solver': 'svd', 'tol': 0.0001}

In [13]:
res1.best_score_

0.7910130718954249

In [14]:
y_test_pred=res1.best_estimator_.predict(test_set)


In [15]:
import sklearn
from sklearn.metrics import accuracy_score
sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100

89.77272727272727

In [16]:
#error rate=100-accuracy
100-(sklearn.metrics.accuracy_score(test_labels, y_test_pred)*100)

10.227272727272734

In [17]:
matthews_corrcoef(test_labels, y_test_pred)*100

78.69121737721238

In [18]:
from sklearn.metrics import precision_score
sklearn.metrics.precision_score(test_labels, y_test_pred, pos_label= 'b')*100

100.0

In [19]:
from sklearn.metrics import recall_score
sklearn.metrics.recall_score(test_labels, y_test_pred, pos_label= 'b')*100

71.875

In [20]:
from sklearn.metrics import f1_score
sklearn.metrics.f1_score(test_labels, y_test_pred,  pos_label= 'b')*100

83.63636363636363